# Modeling: Loan Application Approval
### Comparison of Classification Algorithms Performance Metrics 

In [15]:
conda install -c conda-forge imbalanced-learn

Solving environment: done

## Package Plan ##

  environment location: /Users/Reeves1/opt/anaconda3

  added / updated specs:
    - imbalanced-learn


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.10.1               |   py38h50d1736_0         3.1 MB  conda-forge
    imbalanced-learn-0.8.0     |     pyhd8ed1ab_0         109 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

  imbalanced-learn   conda-forge/noarch::imbalanced-learn-0.8.0-pyhd8ed1ab_0

The following packages will be UPDATED:

  conda                               4.10.0-py38h50d1736_1 --> 4.10.1-py38h50d1736_0



conda-4.10.1         | 3.1 MB    | ##################################### | 100% 
imbalanced-learn-0.8 | 109 KB    | ##################################### | 100% 
P

#### In this notebook I will recreate my baseline model, utilizing a 70/30 train-test split, and attempt to boost the recall score of the minority class via oversampling and undersampling as a method of accounting for the imbalanced classes. I will also utilize a Random Forest, and Decision Tree algorithm to compare performance metrics for each on the training data once it has been oversampled and undersampled. 

In [1]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [2]:
df_clean = pd.read_csv('data/loan_data_clean.csv')

In [7]:
del df_clean['ApplicantIncome']
del df_clean['Loan_ID']
del df_clean['CoapplicantIncome']

In [9]:
dummy_df = df_clean[['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'HouseholdIncome']]

In [10]:
X = pd.get_dummies(dummy_df)
y = df_clean['Loan_Status'].values

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1, stratify=y)

In [12]:
#construct Logistic Regression model
clf = LogisticRegression(max_iter=1000)
#fit the model on the training data
clf.fit(X_train, y_train)
#introduce variable to be reused later 
y_predict_test = clf.predict(X_test)

In [13]:
y_predict_training = clf.predict(X_train)
print("[Training Classification Report]")
print(classification_report(y_train, y_predict_training))

print("[Test Classification Report]")
print(classification_report(y_test, y_predict_test))

[Training Classification Report]
              precision    recall  f1-score   support

           N       0.88      0.43      0.58       134
           Y       0.79      0.97      0.87       295

    accuracy                           0.80       429
   macro avg       0.83      0.70      0.73       429
weighted avg       0.82      0.80      0.78       429

[Test Classification Report]
              precision    recall  f1-score   support

           N       1.00      0.40      0.57        58
           Y       0.78      1.00      0.88       127

    accuracy                           0.81       185
   macro avg       0.89      0.70      0.72       185
weighted avg       0.85      0.81      0.78       185



## Logistic Regression
#### Oversampling the minority class will be the first step in attempting to improve the recall of the baseline model. 

In [18]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [26]:
#construct Logistic Regression model
clf = LogisticRegression(max_iter=1000)
#fit the model on the training data
clf.fit(X_resampled, y_resampled)
#introduce variable to be reused later 
y_predict_test = clf.predict(X_test)

In [21]:
y_predict_training = clf.predict(X_resampled)
print("[Training Classification Report]")
print(classification_report(y_resampled, y_predict_training))

print("[Test Classification Report]")
print(classification_report(y_test, y_predict_test))

[Training Classification Report]
              precision    recall  f1-score   support

           N       0.50      0.74      0.60       295
           Y       0.51      0.27      0.35       295

    accuracy                           0.50       590
   macro avg       0.50      0.50      0.48       590
weighted avg       0.50      0.50      0.48       590

[Test Classification Report]
              precision    recall  f1-score   support

           N       0.27      0.55      0.36        58
           Y       0.60      0.31      0.41       127

    accuracy                           0.38       185
   macro avg       0.43      0.43      0.38       185
weighted avg       0.50      0.38      0.39       185



#### The performance metrics for the baseline model after being oversampled for the minority class performs better in the training Classification report for recall increasing from .43 to .74. We see an increase in recall in the Test set as well, however, in both models the Precision score of the Majority Class is significantly worse. I am curious to see how SMOTE affects the recall score of this model. 

In [22]:
from imblearn.over_sampling import SMOTE

In [23]:
X_resampled1, y_resampled1 = SMOTE().fit_resample(X_train, y_train)
#construct Logistic Regression model
clf = LogisticRegression(max_iter=1000)
#fit the model on the training data
clf.fit(X_resampled1, y_resampled1)
#introduce variable to be reused later 
y_predict_test = clf.predict(X_test)
y_predict_training = clf.predict(X_resampled1)
print("[Training Classification Report]")
print(classification_report(y_resampled1, y_predict_training))

print("[Test Classification Report]")
print(classification_report(y_test, y_predict_test))

[Training Classification Report]
              precision    recall  f1-score   support

           N       0.93      0.76      0.83       295
           Y       0.79      0.94      0.86       295

    accuracy                           0.85       590
   macro avg       0.86      0.85      0.85       590
weighted avg       0.86      0.85      0.85       590

[Test Classification Report]
              precision    recall  f1-score   support

           N       0.76      0.43      0.55        58
           Y       0.78      0.94      0.85       127

    accuracy                           0.78       185
   macro avg       0.77      0.68      0.70       185
weighted avg       0.77      0.78      0.76       185



#### SMOTE provided interesting results! The recall metric in the Training set was significantly higher than the baseline model and the Precision score did not suffer any decline remaining at .79. Precision score for the minority class and recall for the majority class were decreased by .07 and .06 respectively. Performance on the test set was significantly worse than the training set, with the recall for the minority class seeing no improvement. Undersampling will be the final method I attempt on this baseline model.

In [24]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler()
X_resampled2, y_resampled2 = rus.fit_resample(X_train, y_train)

In [25]:
#construct Logistic Regression model
clf = LogisticRegression(max_iter=1000)
#fit the model on the training data
clf.fit(X_resampled2, y_resampled2)
#introduce variable to be reused later 
y_predict_test = clf.predict(X_test)
y_predict_training = clf.predict(X_resampled2)
print("[Training Classification Report]")
print(classification_report(y_resampled2, y_predict_training))

print("[Test Classification Report]")
print(classification_report(y_test, y_predict_test))

[Training Classification Report]
              precision    recall  f1-score   support

           N       0.74      0.67      0.71       134
           Y       0.70      0.77      0.73       134

    accuracy                           0.72       268
   macro avg       0.72      0.72      0.72       268
weighted avg       0.72      0.72      0.72       268

[Test Classification Report]
              precision    recall  f1-score   support

           N       0.49      0.60      0.54        58
           Y       0.80      0.71      0.75       127

    accuracy                           0.68       185
   macro avg       0.64      0.66      0.64       185
weighted avg       0.70      0.68      0.68       185



#### Performance metrics for the minority class recall has seen the best improvement in the test set with an increase from .43 to .60, while simultaneously improving the precision of the majority class in the test set. There was a significant decrease in performance of the precision for the minority class in the test set that cannot be ignored, however. At this point, I have achieved the goal of improving the recall score of the minority class by utilizing undersampling of the training set. Utilizing both oversampling and undersampling, as well as SMOTE on a Random Forest Algorithm will be my next step. 

## Random Forest 

In [27]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier()
ros = RandomOverSampler()
X_resampled3, y_resampled3 = ros.fit_resample(X_train, y_train)
clf.fit(X_resampled3, y_resampled3)
y_predict_test = clf.predict(X_test)
y_predict_training = clf.predict(X_resampled3)
print("[Training Classification Report]")
print(classification_report(y_resampled3, y_predict_training))

print("[Test Classification Report]")
print(classification_report(y_test, y_predict_test))

[Training Classification Report]
              precision    recall  f1-score   support

           N       1.00      1.00      1.00       295
           Y       1.00      1.00      1.00       295

    accuracy                           1.00       590
   macro avg       1.00      1.00      1.00       590
weighted avg       1.00      1.00      1.00       590

[Test Classification Report]
              precision    recall  f1-score   support

           N       0.69      0.57      0.62        58
           Y       0.82      0.88      0.85       127

    accuracy                           0.78       185
   macro avg       0.75      0.73      0.74       185
weighted avg       0.78      0.78      0.78       185



#### The training set results were perfect across the board for the Random Forest model, with the test set being significantly better than the oversampling results on the baseline Log. Regression model. It will be interesting to see the effects of SMOTE on this model as well! 

In [28]:
X_resampled4, y_resampled4 = SMOTE().fit_resample(X_train, y_train)
#construct Logistic Regression model
clf = RandomForestClassifier()
#fit the model on the training data
clf.fit(X_resampled4, y_resampled4)
#introduce variable to be reused later 
y_predict_test = clf.predict(X_test)
y_predict_training = clf.predict(X_resampled4)
print("[Training Classification Report]")
print(classification_report(y_resampled4, y_predict_training))

print("[Test Classification Report]")
print(classification_report(y_test, y_predict_test))

[Training Classification Report]
              precision    recall  f1-score   support

           N       1.00      1.00      1.00       295
           Y       1.00      1.00      1.00       295

    accuracy                           1.00       590
   macro avg       1.00      1.00      1.00       590
weighted avg       1.00      1.00      1.00       590

[Test Classification Report]
              precision    recall  f1-score   support

           N       0.71      0.55      0.62        58
           Y       0.81      0.90      0.85       127

    accuracy                           0.79       185
   macro avg       0.76      0.72      0.74       185
weighted avg       0.78      0.79      0.78       185



#### Overall, this model performs marginally better on the test set than Random Oversampled model with the Precision scores of both classes increasing as well as the recall score of the Majority class. The recall for the minority class performed a little worse, by .02, in this model than the ROS. 

In [29]:
rus = RandomUnderSampler()
X_resampled5, y_resampled5 = rus.fit_resample(X_train, y_train)
#construct Logistic Regression model
clf = RandomForestClassifier()
#fit the model on the training data
clf.fit(X_resampled5, y_resampled5)
#introduce variable to be reused later 
y_predict_test = clf.predict(X_test)
y_predict_training = clf.predict(X_resampled5)
print("[Training Classification Report]")
print(classification_report(y_resampled5, y_predict_training))

print("[Test Classification Report]")
print(classification_report(y_test, y_predict_test))

[Training Classification Report]
              precision    recall  f1-score   support

           N       1.00      1.00      1.00       134
           Y       1.00      1.00      1.00       134

    accuracy                           1.00       268
   macro avg       1.00      1.00      1.00       268
weighted avg       1.00      1.00      1.00       268

[Test Classification Report]
              precision    recall  f1-score   support

           N       0.52      0.60      0.56        58
           Y       0.81      0.75      0.78       127

    accuracy                           0.70       185
   macro avg       0.66      0.68      0.67       185
weighted avg       0.72      0.70      0.71       185



#### Comparing the recall and precision scores for the ROS, RUS, and SMOTE resampling for the Random Forest Model, I believe the best model would be the SMOTE resampled Random Forest. This model performs better than the baseline model as well! Looking at how another algorithm performs will be valuable as well. 

## Decision Tree

In [30]:
from sklearn.tree import DecisionTreeClassifier

In [31]:
clf = DecisionTreeClassifier()
ros = RandomOverSampler()
X_resampled6, y_resampled6 = ros.fit_resample(X_train, y_train)
clf.fit(X_resampled6, y_resampled6)
y_predict_test = clf.predict(X_test)
y_predict_training = clf.predict(X_resampled6)
print("[Training Classification Report]")
print(classification_report(y_resampled6, y_predict_training))

print("[Test Classification Report]")
print(classification_report(y_test, y_predict_test))

[Training Classification Report]
              precision    recall  f1-score   support

           N       1.00      1.00      1.00       295
           Y       1.00      1.00      1.00       295

    accuracy                           1.00       590
   macro avg       1.00      1.00      1.00       590
weighted avg       1.00      1.00      1.00       590

[Test Classification Report]
              precision    recall  f1-score   support

           N       0.49      0.43      0.46        58
           Y       0.75      0.80      0.77       127

    accuracy                           0.68       185
   macro avg       0.62      0.61      0.62       185
weighted avg       0.67      0.68      0.68       185



#### It appears as though the Decision Tree Classifier performance after ROS is closer to the results we saw from the Log. Reg. Model, that being it is not quite as good as the Random Forest Model. 

In [32]:
X_resampled7, y_resampled7 = SMOTE().fit_resample(X_train, y_train)
#construct Logistic Regression model
clf = DecisionTreeClassifier()
#fit the model on the training data
clf.fit(X_resampled7, y_resampled7)
#introduce variable to be reused later 
y_predict_test = clf.predict(X_test)
y_predict_training = clf.predict(X_resampled7)
print("[Training Classification Report]")
print(classification_report(y_resampled7, y_predict_training))

print("[Test Classification Report]")
print(classification_report(y_test, y_predict_test))

[Training Classification Report]
              precision    recall  f1-score   support

           N       1.00      1.00      1.00       295
           Y       1.00      1.00      1.00       295

    accuracy                           1.00       590
   macro avg       1.00      1.00      1.00       590
weighted avg       1.00      1.00      1.00       590

[Test Classification Report]
              precision    recall  f1-score   support

           N       0.50      0.50      0.50        58
           Y       0.77      0.77      0.77       127

    accuracy                           0.69       185
   macro avg       0.64      0.64      0.64       185
weighted avg       0.69      0.69      0.69       185



#### We see a similar performance in this model with SMOTE resampling methods applied, marginally better than the baseline model, but not as good as the Random Forest Model. 

In [33]:
rus = RandomUnderSampler()
X_resampled8, y_resampled8 = rus.fit_resample(X_train, y_train)
#construct Logistic Regression model
clf = DecisionTreeClassifier()
#fit the model on the training data
clf.fit(X_resampled8, y_resampled8)
#introduce variable to be reused later 
y_predict_test = clf.predict(X_test)
y_predict_training = clf.predict(X_resampled8)
print("[Training Classification Report]")
print(classification_report(y_resampled8, y_predict_training))

print("[Test Classification Report]")
print(classification_report(y_test, y_predict_test))

[Training Classification Report]
              precision    recall  f1-score   support

           N       1.00      1.00      1.00       134
           Y       1.00      1.00      1.00       134

    accuracy                           1.00       268
   macro avg       1.00      1.00      1.00       268
weighted avg       1.00      1.00      1.00       268

[Test Classification Report]
              precision    recall  f1-score   support

           N       0.41      0.66      0.50        58
           Y       0.78      0.57      0.66       127

    accuracy                           0.59       185
   macro avg       0.60      0.61      0.58       185
weighted avg       0.67      0.59      0.61       185



#### And with the RUS methods on the Decision Tree algorithm we see again that it is not nearly as effective as the Random Forest Model. 

#### In conclusion, the Random Forest model utilizing SMOTE as a means of resampling the training data sees the best improvement of the recall score on the test set, with the other performance metrics increasing as well. While it is the BEST model of the ones available, it's recall score is still 